In [1]:
! git clone https://github.com/naftalima/androidGames.git

Cloning into 'androidGames'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 311 (delta 1), reused 12 (delta 1), pack-reused 297
Receiving objects: 100% (311/311), 2.00 MiB | 12.02 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [3]:
cd androidGames/

/home/jovyan/binder/androidGames


In [4]:
import pandas as pd
import numpy as np
import math

#df: data frame
df = pd.read_csv('data/_SELECT_name_battery_level_version_name_device_id_timestamp_FROM_201909111900.csv')
# name, battery_level, version_name, device_id, timestamp

# str into  <class 'pandas._libs.tslib.Timestamp>
df['timestamp'] =  pd.to_datetime(df['timestamp'])

df.head(20)

,name,battery_level,version_name,device_id,timestamp
0,com.dts.freefireth,0.89,1.17.1,3383,2018-06-17 03:55:49
1,com.dts.freefireth,0.80,1.17.1,3383,2018-06-17 04:13:19
2,com.dts.freefireth,0.80,1.17.1,3383,2018-06-17 04:13:49
3,com.dts.freefireth,0.79,1.17.1,3383,2018-06-17 04:14:19
4,com.dts.freefireth,0.79,1.17.1,3383,2018-06-17 04:14:49
5,com.dts.freefireth,0.79,1.17.1,3383,2018-06-17 04:15:19
6,com.dts.freefireth,0.78,1.17.1,3383,2018-06-17 04:15:49
7,com.dts.freefireth,0.78,1.17.1,3383,2018-06-17 04:16:19
8,com.dts.freefireth,0.44,1.17.1,3383,2018-06-17 05:17:47
9,com.dts.freefireth,0.44,1.17.1,3383,2018-06-17 05:18:17


In [9]:

def games_names():
    f = open("data/jogos.txt", "r")
    jogos=[]
    for line in f:
        linha = line.split(": ")
        jogos.append([linha[0],linha[1]])
    f.close()
    for  i in range(len(jogos)):
        jogos[i][1] = jogos[i][1].replace('\n','')
    return(jogos)

def url(jogosnomes):
    urls= []
    for i in jogosnomes:
        urls.append(i[1])
    return(urls)

def GamesMostUsed(listOfGames):
    """Os com mais dados do top 20 da playstore"""
    amount = []
    for i in listOfGames:
        qtd=0
        for index, row in df.iterrows():
            if row["name"] == i:
                qtd+=1
        amount.append(qtd)
        # print(i,"qtd: ",qtd)
    gamesGood = []
    for i in range(len(amount)):
        if amount[i] > 1000:
            gamesGood.append(listOfGames[i])
    return(gamesGood)

def agrupando(nome):
    # version_name={ #NESTED DICT
    #     'Version_Name': {'Device_Id': [ [timestamp, battery level] ] }
    #                                        #NESTED LIST
    version_name={0.0: {0: []}}
    for index, row in df.iterrows():
        if row["name"] == nome:  #para todos os dados do jogo selecionado
            #agrupando pela versão
            if not row["version_name"] in version_name:
                version_name[row["version_name"]] = {} # cria
            #agrupando pelo dispositivo
            if not row['device_id'] in version_name[row["version_name"]]:
                version_name[row["version_name"]][row['device_id']] = [] #cria
            version_name[row["version_name"]][row['device_id']].append([row['timestamp'],row['battery_level']]) #insere
    del version_name[0.0]
    return(version_name)

def porUso(jogoAgrupado):
    """Separando por uso e deletando os repetidos"""
    dfJogo = pd.DataFrame()
    dateInit = []
    batteryInit =[]
    dateFinal = []
    batteryFinal =[]
    for key, val in jogoAgrupado.items():
        # print("version_name",key)
        for key0, val0 in val.items():
            # print("device_id", key0)
            n = len(val0)
            for i in range(n-1):
                if  i == 0: #inicio da lista
                    dateInit.append(val0[i][0])
                    batteryInit.append(val0[i][1])
                    # print("START","inicio: ", val0[i][1])
                diff =  val0[i+1][1] - val0[i][1]
                if ( diff < -0.03 ) or ( diff > 0.0): #parou
                    dateFinal.append(val0[i][0])
                    batteryFinal.append(val0[i][1])
                    dateInit.append(val0[i+1][0])
                    batteryInit.append(val0[i+1][1])
                    # print("STOP","fim: ", val0[i][0], "inicio do prox:", val0[i+1][0])
                if i == n-2: ##fim da lista
                    dateFinal.append(val0[i+1][0])
                    batteryFinal.append(val0[i+1][1])
                    # print("END", "final:", val0[i+1][1])
    # print("inicio: ", batteryInit, dateInit)
    # print("fim: ",batteryFinal,dateFinal)
    dfJogo['BatteryLevelFinal'] = batteryFinal
    dfJogo['BatteryLevelInitial'] = batteryInit
    dfJogo['TimestampFinal'] = dateFinal
    dfJogo['TimestampInitial'] = dateInit
    # print(dfJogo.head(10))
    batteryused = []
    elapsedtime = []
    for index, row in dfJogo.iterrows():
        battery = np.subtract(row['BatteryLevelInitial'], row['BatteryLevelFinal'] )
        time = np.subtract( row['TimestampFinal'] , row['TimestampInitial'] )
        # # <class 'pandas._libs.tslib.Timedelta'>
        # # diff.seconds <class 'int'>
        batteryused.append(battery)
        elapsedtime.append(time.seconds)
    dfJogo = dfJogo.drop(columns="BatteryLevelFinal")
    dfJogo = dfJogo.drop(columns="BatteryLevelInitial")
    dfJogo = dfJogo.drop(columns="TimestampInitial")
    dfJogo = dfJogo.drop(columns="TimestampFinal")

    dfJogo['Battery_Used'] = batteryused
    dfJogo['ElapsedTimestamp'] = elapsedtime
    dfJogo = dfJogo.set_index("Battery_Used")
    dfJogo = dfJogo.drop(0, axis=0)

    return(dfJogo)


def exportCSV(gamesList):
    """Exportando cada uso de cada jogo em um csv com nome do jogo"""
    for i in gamesList:
        nameG = i
        dfG = porUso(agrupando(nameG))
        arqName = 'data/' + nameG + '.csv'
        export_csv = dfG.to_csv (arqName, header=True)

def Count_Users_Versions(games_list):
    users_versions = {}
    for i in games_list:
        nameG = i
        dictG = agrupando(nameG)
        versoes = len(dictG)
        users = 0
        for key in dictG: #key == versionname
            users += len(dictG[key])
        users_versions[nameG] = [versoes,users]
    df_uv = pd.DataFrame.from_dict(users_versions,orient='index')
    df_uv.columns = ['versoes','users']
    return(df_uv)

def main_cleaning():
    selected_games = GamesMostUsed(games)
    exportCSV(selected_games)
    return()

def Maior_Menor(dfCountUV):
    print('maior qtd:')
    print(dfCountUV.idxmax())  
    print(dfCountUV.max())
    print('menor qtd:')  
    print(dfCountUV.idxmin())  
    print(dfCountUV.min())
    return()

In [10]:
games =  url(games_names())
print(games)

['com.dts.freefireth', 'com.supercell.clashroyale', 'com.nianticlabs.pokemongo', 'com.igg.android.lordsmobile', 'com.supercell.brawlstars', 'com.supercell.clashofclans', 'com.king.candycrushsaga', 'com.mobile.legends', 'com.tencent.ig', 'com.playrix.gardenscapes', 'com.miniclip.eightballpool', 'com.playrix.township', 'jp.konami.duellinks', 'com.king.candycrushsodasaga', 'com.rovio.baba', 'com.hcg.cok.gp', 'com.king.farmheroessaga', 'com.youmusic.magictiles', 'com.kiloo.subwaysurf', 'com.imangi.templerun2']


In [12]:
qtd = Count_Users_Versions(games)
print(qtd)

                             versoes  users
com.dts.freefireth                24    320
com.supercell.clashroyale         20     73
com.nianticlabs.pokemongo         18     22
com.igg.android.lordsmobile        7      8
com.supercell.brawlstars          11     31
com.supercell.clashofclans        36    144
com.king.candycrushsaga           31     54
com.mobile.legends                45    318
com.tencent.ig                     8     37
com.playrix.gardenscapes           5      6
com.miniclip.eightballpool        15     45
com.playrix.township               6     12
jp.konami.duellinks                6      6
com.king.candycrushsodasaga       12     17
com.rovio.baba                     9     12
com.hcg.cok.gp                     0      0
com.king.farmheroessaga           10     11
com.youmusic.magictiles           15     23
com.kiloo.subwaysurf              33    141
com.imangi.templerun2             23     43


In [13]:
print(Maior_Menor(qtd))

maior qtd:
versoes    com.mobile.legends
users      com.dts.freefireth
dtype: object
versoes     45
users      320
dtype: int64
menor qtd:
versoes    com.hcg.cok.gp
users      com.hcg.cok.gp
dtype: object
versoes    0
users      0
dtype: int64
()


In [17]:
gameMostPopular =  GamesMostUsed(url(games_names()))
print(gameMostPopular)

['com.dts.freefireth', 'com.supercell.clashroyale', 'com.nianticlabs.pokemongo', 'com.supercell.clashofclans', 'com.king.candycrushsaga', 'com.mobile.legends', 'com.tencent.ig', 'com.miniclip.eightballpool', 'com.playrix.township', 'com.king.candycrushsodasaga', 'com.kiloo.subwaysurf', 'com.imangi.templerun2']


In [19]:
qtdMostPopulardf = Count_Users_Versions(gameMostPopular)
print(qtdMostPopulardf)

                             versoes  users
com.dts.freefireth                24    320
com.supercell.clashroyale         20     73
com.nianticlabs.pokemongo         18     22
com.supercell.clashofclans        36    144
com.king.candycrushsaga           31     54
com.mobile.legends                45    318
com.tencent.ig                     8     37
com.miniclip.eightballpool        15     45
com.playrix.township               6     12
com.king.candycrushsodasaga       12     17
com.kiloo.subwaysurf              33    141
com.imangi.templerun2             23     43


In [20]:
print(Maior_Menor(qtdMostPopulardf))

maior qtd:
versoes    com.mobile.legends
users      com.dts.freefireth
dtype: object
versoes     45
users      320
dtype: int64
menor qtd:
versoes    com.playrix.township
users      com.playrix.township
dtype: object
versoes     6
users      12
dtype: int64
()
